In [6]:
import sys
sys.path.append('..')

In [13]:
from apify_client import ApifyClient
from langchain.document_loaders.apify_dataset import ApifyDatasetLoader
from config import APIFY_API_TOKEN

# when I tested this, I needed to put the token here. The environment variable wouldn't do:
apify = ApifyClient(
    token=APIFY_API_TOKEN
)
actor = apify.actor(
    actor_id="apify/website-content-crawler",
)

In [25]:
# initiate run and wait to finish:
client_run = actor.call(
    run_input={"startUrls": [{"url": "https://docs.smith.langchain.com/overview"}]},
)
# https://python.langchain.com/en/latest/

In [26]:
dataset_id = client_run.get("defaultDatasetId")
print(dataset_id)  # you can also find the ids at https://console.apify.com/storage/datasets

NpCG9x1sGpBoXK1zV


In [27]:
from langchain.schema import Document

loader = ApifyDatasetLoader(
    dataset_id=dataset_id,
    dataset_mapping_function=lambda dataset_item: Document(
        page_content=dataset_item["text"], metadata={"source": dataset_item["url"]}
    ),
)

In [30]:
persist_directory = os.path.expanduser("~") + "/Documents/indexes/langsmith"

In [31]:
persist_directory

'/Users/ben/Documents/indexes/langsmith'

In [32]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma

# persist index to disk and load it up again
index = VectorstoreIndexCreator(
    vectorstore_cls=Chroma,
    vectorstore_kwargs={
        "persist_directory": persist_directory
    }).from_loaders([loader])

In [33]:
# Finally, query the vector database:
query = "What is LangSmith?"
result = index.query_with_sources(query)
print(result["answer"])
print(result["sources"])

 LangSmith is a tool used to debug LLMs, chains, and agents, track token usage, and collect examples of unexpected outcomes.

https://docs.smith.langchain.com/overview


In [42]:
from langchain.embeddings import OpenAIEmbeddings

# we need embeddings for retrieval
embeddings = OpenAIEmbeddings()

In [43]:
# load the index back up:
db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [46]:
from langchain.chains import FlareChain
from langchain.chat_models import ChatOpenAI

LLM = ChatOpenAI(
    model_name="gpt-4",
    temperature=0,
)
agent = FlareChain.from_llm(
    llm=LLM,
    retriever=db.as_retriever(),
    max_generation_len=1500,
    verbose=True,
    max_tokens_limit=1500,
)

In [47]:
agent.run("What is LangSmith and what can it do?")



> Entering new FlareChain chain...
Current Response: 
Prompt after formatting:
Respond to the user message using any relevant context. If context is provided, you should ground your answer in that context. Once you're done responding return FINISHED.

>>> CONTEXT: 
>>> USER INPUT: What is LangSmith and what can it do?
>>> RESPONSE: 
Generated Questions: ['What are some of the tools and resources that LangSmith offers to aid in language learning?', '"What is one of the features that LangSmith offers to aid in language learning?"', '"What feature does LangSmith provide to help you understand your learning progress and areas for improvement?"']

> Finished chain.


' LangSmith is a platform for building, debugging, and monitoring natural language processing (NLP) applications. It provides a suite of tools for debugging LLMs, chains, agents, tools, and retrievers, as well as for monitoring performance and collecting feedback. It also makes it easy to curate datasets and export them for use in other contexts. '